In [1]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_model
from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood
from botorch.optim import optimize_acqf
from botorch.acquisition.multi_objective import qExpectedHypervolumeImprovement
from botorch.utils.multi_objective import infer_reference_point
from botorch.utils.sampling import sample_simplex

# Define your objective functions
def objective_function(x):
    # This is where you define your multi-objective function
    # For example, two objectives based on input tensor x
    objective1 = -torch.norm(x - 0.5, dim=-1)
    objective2 = torch.sum(x, dim=-1)
    return torch.stack([-objective1, objective2], dim=-1)

# Generate some random data
# For example, 10 data points in 2D space
train_x = torch.rand(10, 2)
train_y = objective_function(train_x)

# Fit a GP model
model = SingleTaskGP(train_x, train_y)
mll = ExactMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_model(mll)

# Infer the reference point
reference_point = infer_reference_point(train_y)

# Define the acquisition function
qEHVI = qExpectedHypervolumeImprovement(
    model=model,
    ref_point=reference_point.tolist(),  # reference point for hypervolume calculation
    sampler=sample_simplex,  # sampler for the qEHVI acquisition function
    partitioning=N
)

# Optimize the acquisition function
new_candidate, _ = optimize_acqf(
    acq_function=qEHVI,
    bounds=torch.tensor([[0.0, 0.0], [1.0, 1.0]]),
    q=1,
    num_restarts=5,
    raw_samples=20,
)

print(new_candidate)


c:\Users\springnuance\AppData\Local\Programs\Python\Python311\Lib\site-packages\botorch\models\gpytorch.py:129: UserWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  warnings.warn(_get_single_precision_warning(X.dtype), UserWarning)
c:\Users\springnuance\AppData\Local\Programs\Python\Python311\Lib\site-packages\botorch\models\utils\assorted.py:201: InputDataWarning: Input data is not standardized. Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


TypeError: qExpectedHypervolumeImprovement.__init__() missing 1 required positional argument: 'partitioning'